# Full notebook for combining labels

In [5]:
import numpy as np
#import napari
from skimage.io import imread, imsave
from skimage import img_as_uint
from timeit import default_timer as time
from pathlib import Path

from napari_segmentation_overlap_filter.functions import return_overlap_segmentation, return_non_zero_pixels_from_Image, overlapping_objects, get_object_sizes, create_overlap_df, remove_objects, combine_images, check_dask, pct_overlap_filter

### Utility Functions

In [6]:
def read_and_convert_to_16bit(image_path):
    # Read the image
    image = imread(image_path)

    # Check if the image is 16-bit
    if image.dtype != 'uint16':
        image = np.uint16(image)

    return image

### Variables

If `default` is set `True` then will use default variables for paths and filename pattern matching, otherwise specify new variables

In [7]:
default = False

- `root_dir_path` is the directory containing all individual folders with respective Trackmate .xml files and resultant label .tif files
- `image_dir_pattern` find all folders containing labels with this pattern name
- `label_img_pattern` find all label images in above folder with this pattern name
- `output_folder` path to folder you want combined labels to end up, default is same folder containing labels

In [15]:
if default is True:
    root_dir_path = Path.cwd()
    image_dir_pattern = 'c2*'
    label_img_pattern = 'LblImg_*_pixels.tif'

In [35]:
# specify own pattern matching rules

if default is False: 
    root_dir_path = Path('/nemo/stp/lm/working/shared/vd_tf/Training_Data/Images')
    image_dir_pattern = 'C3*'
    label_img_pattern = 'LblImg_*_pixels.tif'
    output_dir_path = Path('/nemo/stp/lm/working/shared/vd_tf/Training_Data/Images/results')

In [36]:
print(f'Root directory is: {root_dir_path}')
print(f'Pattern for label folder names is: {image_dir_pattern}')
print(f'Pattern for label image filenames: {label_img_pattern}')

if default is False:
    print(f'Output results directory is: {output_dir_path}')

Root directory is: /nemo/stp/lm/working/shared/vd_tf/Training_Data/Images
Pattern for label folder names is: C3*
Pattern for label image filenames: LblImg_*_pixels.tif
Output results directory is: /nemo/stp/lm/working/shared/vd_tf/Training_Data/Images/results


### Finding folders contaning labels

In [31]:
# Finding all Channel 2 folders with name containing 'c2'
root_dir = root_dir_path
image_dirs = [item for item in root_dir.glob(image_dir_pattern) if item.is_dir()]
image_dirs

[PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTHT3-2b_170-219'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTGX25-3b_81-130C'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTHT3-1h_281-330'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTGX74-1e_201-250C'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTGX78-2h_221-270'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTGX76-1m_25-74C'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTHT4-3g_300-349'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTHT1-2k_80-129'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTHT3-3g_254-303'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTGX76-2e_147-196')]

### Look at label images in first folder

In [32]:
# Get only label images in folder with filename matching 'LblImg_*_pixels.tif'
first_folder = image_dirs[0]
labels = [item for item in first_folder.glob(label_img_pattern)]
labels

[PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTHT3-2b_170-219/LblImg_C3-JTHT3-2b_170-219_60_pixels.tif'),
 PosixPath('/nemo/stp/lm/working/daov/misc_projects/new_segmentation/C3-JTHT3-2b_170-219/LblImg_C3-JTHT3-2b_170-219_15_pixels.tif')]

### List of pixel sizes to combine

In [33]:
# Provide pixel sizes you want to combine, should be in filename of label images i.e. LblImg_metadata_15_pixels.tif
all_pixel_sizes = [[15, 60],
                   # [15, 75],
              ]

### Combining labels

Will iterate through each folder within `root_dir` which follow pattern specified by `image_dir_pattern` and find all label images specificied by filename following `label_img_pattern` then combine labels according to `all_pixel_sizes` and output resulting combined segmentation maps to the existing labels folder or otherwise specifed by `output_dir_path`

In [34]:
# Get images with specified pixel sizes
for folder in image_dirs:

    print(f'Looking in label folder {folder.stem}')
    
    for pixel_sizes in all_pixel_sizes:
        sorted_sizes = sorted(pixel_sizes)
        small_size = sorted_sizes[0]
        large_size = sorted_sizes[1]

        # All label images in folder 
        all_labels = [str(image) for image in folder.glob(label_img_pattern)]

        # Looking for specific labels with specified pixel sizes
        list_small_image = [i for i in all_labels if str(sorted_sizes[0])+'_pix' in i]
        list_large_image = [i for i in all_labels if str(sorted_sizes[1])+'_pix' in i]

        # If folder does not have label images of specified pixel sizes, move on to next combination
        if len(list_small_image) < 1:
            print(f'No labelled images of {small_size} pixel found in {folder.stem}')
            continue
        elif len(list_large_image) < 1:
            print(f'No labelled images of {large_size} pixel found in {folder.stem}')
            continue
        else: 
            print(f'Found {small_size} and {large_size} pixel labelled images')
        
        Image_A_path = Path(list_small_image[0])
        Image_B_path = Path(list_large_image[0])
        
        Image_A = read_and_convert_to_16bit(Image_A_path)
        Image_B = read_and_convert_to_16bit(Image_B_path)
    
        # Finding overlapping objects
        print(f'Finding overlapping objects for {small_size} and {large_size} pixel sizes')
        df = create_overlap_df(Image_A, Image_B)
    
        # Filtering overlapping objects based on percent overlap
        df_filtered = pct_overlap_filter(df, 0)
    
        # Removing objects in Image_A which overlap with Image_B based on percent overlap
        dask_bool = check_dask()
        print(f'Removing overlapping objects from {Image_A_path.stem}')
        Image_A_overlap_removed = remove_objects(Image_A, list(df_filtered['Image_A_Object_ID']), dask_bool)
    
        # Combining images
        print('Combining labels')
        Image_Combined = combine_images(Image_A_overlap_removed, Image_B)
    
        # Saving combine image
        print('Saving combined labels image')

        if default:
            combined_folderpath = folder
        else:
            combined_folderpath = Path(output_dir_path)
            
        combined_filename = 'LblImg_' + folder.parts[-1] + '_' + str(small_size) + '_' + str(large_size) + '_combined.tif'
            
        final_path = Path(combined_folderpath, combined_filename)
        
        imsave(final_path, Image_Combined)
        print(f'Saved combined labels as {final_path.name}')

Looking in label folder C3-JTHT3-2b_170-219
Found 15 and 60 pixel labelled images
Finding overlapping objects for 15 and 60 pixel sizes
create_overlap_df() executed in 0.534586 s
Removing overlapping objects from LblImg_C3-JTHT3-2b_170-219_15_pixels
dask found, using dask for parallelisation of removing overlapping objects
remove_objects() executed in 0.053738 s
Combining labels
Saving combined labels image
Saved combined labels as LblImg_C3-JTHT3-2b_170-219_15_60_combined.tif
Looking in label folder C3-JTGX25-3b_81-130C
Found 15 and 60 pixel labelled images


/tmp/slurm_3936014/ipykernel_67175/725739988.py:62: UserWarning: /nemo/stp/lm/working/daov/misc_projects/new_segmentation/results/LblImg_C3-JTHT3-2b_170-219_15_60_combined.tif is a low contrast image
  imsave(final_path, Image_Combined)


Finding overlapping objects for 15 and 60 pixel sizes
create_overlap_df() executed in 0.480257 s
Removing overlapping objects from LblImg_C3-JTGX25-3b_81-130C_15_pixels
dask found, using dask for parallelisation of removing overlapping objects
remove_objects() executed in 0.055451 s
Combining labels
Saving combined labels image
Saved combined labels as LblImg_C3-JTGX25-3b_81-130C_15_60_combined.tif
Looking in label folder C3-JTHT3-1h_281-330
Found 15 and 60 pixel labelled images


/tmp/slurm_3936014/ipykernel_67175/725739988.py:62: UserWarning: /nemo/stp/lm/working/daov/misc_projects/new_segmentation/results/LblImg_C3-JTGX25-3b_81-130C_15_60_combined.tif is a low contrast image
  imsave(final_path, Image_Combined)


Finding overlapping objects for 15 and 60 pixel sizes
create_overlap_df() executed in 0.356958 s
Removing overlapping objects from LblImg_C3-JTHT3-1h_281-330_15_pixels
dask found, using dask for parallelisation of removing overlapping objects
remove_objects() executed in 0.054614 s
Combining labels
Saving combined labels image
Saved combined labels as LblImg_C3-JTHT3-1h_281-330_15_60_combined.tif
Looking in label folder C3-JTGX74-1e_201-250C
Found 15 and 60 pixel labelled images


...

/tmp/slurm_3936014/ipykernel_67175/725739988.py:62: UserWarning: /nemo/stp/lm/working/daov/misc_projects/new_segmentation/results/LblImg_C3-JTHT1-2k_80-129_15_60_combined.tif is a low contrast image
  imsave(final_path, Image_Combined)


create_overlap_df() executed in 0.580313 s
Removing overlapping objects from LblImg_C3-JTHT3-3g_254-303_15_pixels
dask found, using dask for parallelisation of removing overlapping objects
remove_objects() executed in 0.053900 s
Combining labels
Saving combined labels image
Saved combined labels as LblImg_C3-JTHT3-3g_254-303_15_60_combined.tif
Looking in label folder C3-JTGX76-2e_147-196
Found 15 and 60 pixel labelled images
Finding overlapping objects for 15 and 60 pixel sizes


/tmp/slurm_3936014/ipykernel_67175/725739988.py:62: UserWarning: /nemo/stp/lm/working/daov/misc_projects/new_segmentation/results/LblImg_C3-JTHT3-3g_254-303_15_60_combined.tif is a low contrast image
  imsave(final_path, Image_Combined)


create_overlap_df() executed in 0.462179 s
Removing overlapping objects from LblImg_C3-JTGX76-2e_147-196_15_pixels
dask found, using dask for parallelisation of removing overlapping objects
remove_objects() executed in 0.054903 s
Combining labels
Saving combined labels image
Saved combined labels as LblImg_C3-JTGX76-2e_147-196_15_60_combined.tif


/tmp/slurm_3936014/ipykernel_67175/725739988.py:62: UserWarning: /nemo/stp/lm/working/daov/misc_projects/new_segmentation/results/LblImg_C3-JTGX76-2e_147-196_15_60_combined.tif is a low contrast image
  imsave(final_path, Image_Combined)
